In [14]:
# Step 1: get all data
from main import setup
dataframes, effect_data = setup("", True)

Rating formular for following section

$\text{Strength Metric} = \text{AvgDmg} + \sum_{i} r_i \times d_i - \text{CD}$


In [15]:
# Step 2: ability rating
import pandas as pd
import numpy as np

# Step 2.1: calculate dice damage (average, max, min, sd)
action_df = dataframes["actions"]
def parse_dice(dice_str):
    """Parses dice in format 'XdY+Z' or 'XdY-Z' and returns a tuple (X, Y, Z)."""
    if '+' in dice_str:
        X, rest = dice_str.split('d')
        Y, Z = rest.split('+')
        return int(X), int(Y), int(Z)
    elif '-' in dice_str:
        X, rest = dice_str.split('d')
        Y, Z = rest.split('-')
        return int(X), int(Y), -int(Z)
    else:
        X, Y = dice_str.split('d')
        return int(X), int(Y), 0

def combined_damage_statistics(damage_entries):
    total_min_damage = 0
    total_max_damage = 0
    total_avg_damage = 0
    total_variance = 0

    for damage in damage_entries:
        if isinstance(damage, list):
            dice_str = damage[0]
        else:
            dice_str = damage
        X, Y, Z = parse_dice(dice_str)

        min_damage = X + Z
        max_damage = X * Y + Z
        avg_damage = (X * (Y + 1) / 2) + Z
        variance = X * (((Y ** 2) - 1) / 12)

        total_min_damage += min_damage
        total_max_damage += max_damage
        total_avg_damage += avg_damage
        total_variance += variance

    combined_std_dev = np.sqrt(total_variance)
    return total_min_damage, total_max_damage, total_avg_damage, combined_std_dev

combined_stats_list = []

for index, row in action_df.iterrows():
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    action_df.at[index, 'min_damage'] = min_dmg
    action_df.at[index, 'max_damage'] = max_dmg
    action_df.at[index, 'average_damage'] = avg_dmg
    action_df.at[index, 'std_dev'] = round(std_dmg, 2)

# Step 2.2: Calculate effect score based on ratings and durations
def calculate_effect_score(effects):
    effect_score = 0
    for effect in effects:
        effect_name = effect[0]
        duration = effect[1] if len(effect) > 1 else 1
        rating = effect_data.get(effect_name, 0)  # Default to 0 if the effect is not found
        effect_score += rating * (int(duration[:-1]) if isinstance(duration, str) and duration[-1] == 'r' else 1)
    return effect_score

# Step 2.3: Calculate a penalty for cooldowns
def calculate_cooldown_penalty(cooldown):
    if isinstance(cooldown, list) and len(cooldown) > 0:
        return int(cooldown[0])
    return 0

# Step 2.4: Calculate a final metric based on all factors
def calculate_strength_metric(row):
    damage_score = row['average_damage']
    effect_score = calculate_effect_score(row['effects'])
    cooldown_penalty = calculate_cooldown_penalty(row['cooldown'])
    
    # Final strength metric formula, can be adjusted for weightage between components
    #TODO adjust the cooldown_penalty to punish more 
    strength_metric = damage_score + effect_score - cooldown_penalty
    return strength_metric

# Apply the calculation to the DataFrame
action_df['strength_metric'] = action_df.apply(calculate_strength_metric, axis=1)
action_df

# current calc looks like this: 

ZeroDivisionError: float division by zero

In [13]:
# Step 3: calculate strength of enemy

,name,legendary,cooldown,range,aoe,hitbonus,savereq,damage,effects,min_damage,max_damage,average_damage,std_dev,strength_metric
0,Abyssaler Nebelhauch,False,"[3, r]",70,radius,0,"[con, 19]","[[12d6, acid]]","[[exhaustion, lv4]]",12.0,72.0,42.0,5.92,44.0
1,Beilhieb,False,[],5,single,5,[],"[[2d8, slashing]]",[],2.0,16.0,9.0,3.24,9.0
2,Actionsname,False,"[1, d]",40,line,9,[],"[[3d8+6, slashing]]","[[prone, 1r]]",9.0,30.0,19.5,3.97,21.5
3,Blendgranate,False,"[1, d]",30,radius,0,"[con, 16]",[],"[[blinded, 1r]]",0.0,0.0,0.0,0.00,2.0
4,Blitz aus der Tiefe,False,"[3, r]",100,single,0,"[dex, 18]","[[2d10, lightning]]","[[paralysed, 2r]]",2.0,20.0,11.0,4.06,18.0
5,Bohrschnabel,False,[],5,single,5,[],"[[1d6+2, piercing]]",[],3.0,8.0,5.5,1.71,5.5
6,Brutale Hiebe,False,[],5,single,10,[],"[[2d12+6, slashing], [2d12+6, slashing]]","[[bleeding, lv2]]",16.0,60.0,38.0,6.90,43.0
7,Drachenstrahl,False,"[3, r]",40,line,6,[],"[[3d8, force]]","[[petrified, 3r]]",3.0,24.0,13.5,3.97,25.5
8,Fischklauen,False,[],10,cone,6,[],"[[2d10, slashing]]","[[bleeding, lv1]]",2.0,20.0,11.0,4.06,16.0
9,Fluch,False,[],35,radius,0,"[int, 17]","[[2d12, necrotic]]","[[petrified, 3r]]",2.0,24.0,13.0,4.88,28.0
